# 1 Connection à Weights and Biases

Pour vous connecter à l'API de Weights and Biases (WandB), utilisez la méthode `login`. Cette dernière vous donnera un lien pour créer un compte WandB, puis vous tomberez sur une page vous indiquant la clé API qu'il faudra coller dans la fenêtre qui apparaît à l'écran pour vous y connecter la première fois. 

Tant que le kernel n'aura pas été réinitialisé, votre session restera active et vous pourrez utiliser librement WandB. Cependant, si vous tentez de poursuivre le cours après avoir réinitalisé le kernel, il sera nécessaire de ré-exécuter la cellule ci-dessous et de coller la clé API de nouveau. 

In [3]:
# Log in to your W&B account
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

De manière similaire à MLflow, WandB est un outil de Tracking des modèles de machine Learning reposant sur la notion de `Runs` et de `Project` (l'équivalent des `Experiment` de MLflow). 

Une `Run` correspond à l'entraînement d'un modèle, avec une combinaison d'hyperparamètres, sur un jeu de données... Et un `Project` est l'ensemble des `Runs` ayant été produites sur une problématique particulère. 

Ces notions nous serons également utile pour naviguer dans l'interface graphique de WandB. 

Exécuter la cellule suivante pour créer une run `My first run` au sein d'un project `classification-car-accidents`. Puisqu'il s'agit de votre première run, un nouveau projet sera créé automatiquement.

In [6]:
# 1. Start a W&B Run
run = wandb.init(
    project="classification-car-accidents",
    name='My first run',
    tags=["baseline", "random-forest"],
)

A présent, nous souhaitons tracker les paramètres du modèle que nous allons entraîner. Pour cela, nous allons créer un dictionnaire `params` dont les clés seront les noms des paramètres et les valeurs seront les valeurs correspondantes. 

Ce dictionnaire doit être intégré dans les configs de WandB pour être tracké.

In [9]:
#  2. Capture a dictionary of hyperparameters
params = {"n_estimators": 100, "criterion": 'gini', "max_depth": 10}

wandb.config = params

Nous pouvons désormais passer à l'étape de l'entraînement du modèle. Le preprocessing ayant déjà été réalisé, il ne nous reste qu'à importer la donnée et entraîner notre modèle (ici Random Forest).

In [10]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
import numpy as np

X_train = pd.read_csv('../data/preprocessed/X_train.csv')
X_test = pd.read_csv('../data/preprocessed/X_test.csv')
y_train = pd.read_csv('../data/preprocessed/y_train.csv')
y_test = pd.read_csv('../data/preprocessed/y_test.csv')
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

rf_classifier = RandomForestClassifier(**params)

#--Train the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

Suite à cet entraînement, nous souhaitons garder une trace des performances de notre modèle, nous allons alors utiliser la méthode wandb.log pour tracker les métriques du modèle que nous venons d'entraîner. 

Tout comme les paramètres du modèle, les métriques doivent être passer dans un dictionnaire. 

In [11]:
train_accuracy = rf_classifier.score(X_train, y_train)
test_accuracy = rf_classifier.score(X_test, y_test)
wandb.log({"train_accuracy": train_accuracy, "test_accuracy": test_accuracy})

Pour finir, il est possible de stocker les artifacts du modèle dans WandB. Il suffit pour cela d'utiliser la méthode `log_artifact` et de préciser le chemin vers l'artifact du modèle. 

In [14]:
import joblib
# #--Save the trained model to a file
model_filename = '../models/trained_model.joblib'
joblib.dump(rf_classifier, model_filename)
# print("Model trained and saved successfully.")

wandb.log_artifact(model_filename)

<Artifact run-wkyd1uss-trained_model.joblib>

# 2 Deuxième expérience

# 3 Expérience groupée